In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1번
철강데이터 종속변수 : target     
데이터 출처 : https://www.kaggle.com/uciml/faulty-steel-plates    
데이터 경로 : /kaggle/input/adp-kr-p4/problem1.csv


### 1-1  EDA(탐색적 데이터 분석)을 하시오
(시각화와 통계량을 제시할 것)

In [ ]:
df = pd.read_csv('/kaggle/input/adp-kr-p4/problem1.csv')
df.info()
import matplotlib.pyplot as plt
df.plot(kind='box',subplots=True,layout = (4,9),figsize=(20,10))
plt.tight_layout()
plt.show()

pd.set_option('display.max_columns',30)
df.head()

In [ ]:
print('numeric 타입이지만 범주형 변수의 경우 아래와 같은 분포를보인다')
for col in df.columns:
    target = df[col]
    if target.nunique() <=10:
        print(col,target.unique())
        display(target.value_counts().to_frame())
        print()
        

### 1-2 변수 선택(VIF), 파생변수 생성, 데이터 분할(train/test(20%))
(시각화와 통계량을 제시할 것)

파생변수로 X_Minimum와 X_Maximum의 평균값을 av 컬럼에 만든다

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

df['av']  = (df['X_Minimum'] + df['X_Maximum'])/2
value ='+'.join(list(df.drop(columns=['target']).columns))
model = ols(f'target ~ {value}', df)
model.exog_names


vif = pd.DataFrame({'col': col, 'VIF': variance_inflation_factor(model.exog, i)} 
             for i, col in enumerate(model.exog_names[1:])).sort_values('VIF',ascending=False)

vif

vif값이 10 미만인 Outside_Global_Index,Maximum_of_Luminosity , Length_of_Conveyer, LogOfAreas, Edges_Y_Index, TypeOfSteel_A300, Outside_X_Index, Empty_Index, Edges_Index, X_Minimum, av 정도가 다중공선성이 없는 변수로 판단한다

In [ ]:
df2 = df.copy()[list(vif[vif.VIF<10].col.values)+['target']]

from sklearn.model_selection import train_test_split

X = df2.drop(columns=['target'])
y = df2['target']

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=32,test_size=0.2,stratify=y)



### 1-3 종속변수들중 "1"인지 아닌지 판단하려한다. 종속변수를 1과 1이 아닌 값(이항)으로 치환하고 로지스틱 회귀 분석을 실시하라.
confusionMatrix를 확인하고 최적의 cut off value 정하여라.

In [ ]:
y_train_l = y_train.copy()
y_test_l = y_test.copy()

y_train_l =y_train_l.map(lambda x : 1 if x==1 else 0)
y_test_l =y_test_l.map(lambda x : 1 if x==1 else 0)


from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear')
model.fit(X_train.values, y_train_l.values)


pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)
from sklearn.metrics import classification_report,roc_curve

print(classification_report(y_test_l,pred))

In [ ]:
import numpy as np
fpr, tpr, thresholds = roc_curve(y_test_l.values, y_prob[:,1],pos_label=1)
J = tpr - fpr
ix = np.argmax(J)
cutoff = thresholds[ix]
cutoff

### 1-4 종속변수(y)를 다항(7 class)인 상태에서 SVM을 포함하여 3가지 알고리즘으로 평가하라.
각 모델에서 confusionMatrix를 확인하고 최적의 cut off value 를 정하여라. (? cut off value 구하는데 뭔가 문제가 있는데,, 문제 오류일까요?)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


s = SVC(probability=True)
rf = RandomForestClassifier(random_state=43)
d = DecisionTreeClassifier(random_state=43)


def check_model(model):
    
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    print(classification_report(y_test,pred))

    
for name,model in [('svm',s),('randomforest',rf),('decision tree',d)]:
    print(name)
    check_model(model)
    print()

### 1-5 종속변수를 제외한 나머지 데이터를 바탕으로 군집분석을 실시하고 최적의 군집수와 군집 레이블을 구하여라. 
군집레이블을 추가한 데이터를 1-4에서 만든 모델중 가장 성능이 좋았던 하나의 모델에 다시 학습하여 F1-score를 비교하라 

In [ ]:
from sklearn.model_selection import train_test_split

X = df2.drop(columns=['target'])
y = df2['target']

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=32,test_size=0.2,stratify=y)



In [ ]:
from sklearn.cluster import KMeans

lst = []
for k in range(1,10):
    model = KMeans(n_clusters=k)
    model.fit(X_train)
    lst.append(model.inertia_)

# Plot ks vs inertias
plt.figure(figsize=(4, 4))

plt.plot(range(1,10), lst, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(range(1,10))
plt.show()

#최고 레이블수 3개

In [ ]:
clust_model = KMeans(n_clusters = 3 , random_state = 42  , algorithm='auto' )
values = clust_model.fit_predict(X_train)  

X_train_cluster = X_train.reset_index(drop=True).copy()
X_test_cluster = X_test.reset_index(drop=True).copy()

X_train_cluster.loc[:,'cluster'] = values
pred = clust_model.predict(X_test) 
X_test_cluster.loc[:,'cluster'] = pred



rf = RandomForestClassifier(random_state=43)
rf.fit(X_train_cluster,y_train)
pred = rf.predict(X_test_cluster)
y_prob = rf.predict_proba(X_test_cluster)
print(classification_report(y_test,pred))

# f1-score값이 증가한 레이블도 있고 감소한 레이블도 존재한다. accuracy값은 동일하다

# 2번 전력데이터
데이터 출처 : 직접제작      
데이터 설명 : 2050년 1년동안의 5유형(A,B,C,D,E)의 전력사용량을 나타낸다. 각유형의 전력사용량은 1분마다 갱신되며 그 값은 누적된다.        
6시간이 지나면(00:00, 06:00, 12:00, 18:00시에) 전력사용량은 0으로 초기화 된다.

- /kaggle/input/adp-kr-p4/problem2_usage.csv   
6시간 간격의 총 전력사용량의 데이터이다. timestamp순서는 섞여있다.      
6시간 간격의 특정 시간대(마지막시각 '05:59','11:59','17:59','23:59')의 전력 총합을 나타낸다.     
데이터의 총합을 구해서 비교할때 부동소수점 오류가 날수 있다. 파이썬의 경우 round(4)를 취하여 해결한다.    


- /kaggle/input/adp-kr-p4/problem2_usage_history.csv   
1분간격의 A,B,C,D,E 유형의 소비 누적 전력을 나타낸다. 같은 6시간간격의 시간대의 데이터는 같은 "6hour_index"값을 가진다.    
00:00, 06:00, 12:00, 18:00시에는 5유형의 전력은 초기화 된다.     
데이터의 총합을 구해서 비교할때 부동소수점 오류가 날수 있다. 파이썬의 경우 round(4)를 취하여 해결한다.     
 

- /kaggle/input/adp-kr-p4/problem2_avg_tem.csv    
2050년 1년동안 일자별 평균 온도를 나타낸다     

### 2-1 usage의 총사용량을 아래와 같은 모양으로 연월별 총합으로 계산하여 CSV 파일로 작성하시오.    
- 일자별 총사용량은 누적사용량이 갱신되기 직전의 최대값들의 합으로 계산한다     
- ['05:59','11:59','17:59','23:59'] 시간대의 A,B,C,D,E의 컬럼별 총합이 각 유형의 일일 사용량이다     

![image](https://github.com/Datamanim/datarepo/blob/main/adp/p4/p2.png?raw=true)

In [ ]:
us = pd.read_csv('/kaggle/input/adp-kr-p4/problem2_usage.csv')
ush = pd.read_csv('/kaggle/input/adp-kr-p4/problem2_usage_history.csv')
us['time'] = pd.to_datetime(us.timestamp, unit='s')
us = us.sort_values('time').reset_index(drop=True)

s = ush[ush['hh:mm'].isin(['05:59','11:59','17:59','23:59'])]
s2 = s.copy()
s2.loc[:,'t'] = s.iloc[:,2:].sum(axis=1).round(4)

zz = pd.merge(ush,pd.merge(s2.rename(columns={'t':'usage'}).reset_index(drop=True),us)[['6hour_index','time']])
zz['time'] = pd.to_datetime(zz['time'])

q = zz[zz['hh:mm'].isin(['05:59','11:59','17:59','23:59'])].copy()
q.loc[:,'date'] = q['time'].dt.strftime('%Y-%m')
q.groupby('date').sum()

### 2-2 가로축을 요일(일~월) 세로축을 평균 전력사용량으로 하는 그래프를 그려라. 각 유형별로 색을 다르게 표현하여 5개의 line plot을 그리며 범례를 표시하라

In [ ]:
q['day'] = q['time'].dt.day_name()
daydf = q.groupby(['day']).mean().reindex(['Sunday', 'Monday', 'Tuesday','Wednesday','Thursday','Friday','Saturday'])
daydf.plot()

### 2-3 요일별 각 유형의 평균 전력 사용량 간에 연관성이 있는지 검정하라

In [ ]:
daydf

In [ ]:
from scipy.stats import chi2_contingency
# print(chi2_contingency(daydf))
chi2 , p ,dof, expected = chi2_contingency(daydf)
print(p)


# p-value 결과 귀무가설을 기각 할 수 없다. 두 변수는 독립적이라 볼 수 있다.

### 2-4 일자(매일)마다 각 유형의 전력사용량의 합을 데이터프레임으로 구하고 일자 데이터에서의 유형별 온도와의 상관계수를 각각 구하여라

In [ ]:
days = q.groupby(q.time.dt.date).sum().reset_index().rename(columns={'time':'date'})
days['date'] = pd.to_datetime(days['date'])
t = pd.read_csv('/kaggle/input/adp-kr-p4/problem2_avg_tem.csv')
t['date'] = pd.to_datetime(t['date'])

pd.merge(t,days).corr().iloc[0,1:].to_frame()